## Analysis of Coronavirus (COVID-19) Pandemic Data 

The purpose of the notebook is to evaluate the coronavirus (COVID-19) data provided by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University (JHU).  This workbook only evaluates the time series dataset provided by JHU-CSSE.

As of March 17, 2020 this notebook is still a work in progress.  Most disappointing is the updates from the JHU dataset for U.S. county reporting seem to have been discontinued. All the datasets from JHU seem to lag behind state reporting.

#### Data Source: 
CSSE-JHU COVID-19 data repository https://github.com/CSSEGISandData/COVID-19  

---

#### Python Libraries

In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt

import requests
import numpy as np
%matplotlib inline
import seaborn as sns

/Users/mark/anaconda3/envs/sandbox2/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


*** The warning message above is the result of the seaborn package needing to be updated.

In [2]:
#plt.style.use('ggplot')
plt.style.use('fivethirtyeight')

#### Load csv data into dataframes

In [5]:
# confirmed cases data
confirm_url = 'https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

# recovered cases data
#recover_url = 'https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered-global.csv'

# death cases data
deaths_url = 'https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'

# create dataframes
confirm_df = pd.read_csv(confirm_url)
recover_df = pd.read_csv(recover_url)
deaths_df = pd.read_csv(deaths_url)


HTTPError: HTTP Error 404: Not Found

In [ ]:
state_abbr_fips_df = pd.read_csv('state_abbr_fips.csv', header=None, dtype=object, names=['NAME','state_fips','state_abbr'])
state_abbr_fips_df.head(15)

#### Set dataframe window viewing size

In [ ]:
pd.set_option('display.max_rows', 1000)

In [ ]:
confirm_df.head(15)

In [ ]:
recover_df.head(15)

In [ ]:
deaths_df.head(15)

### US & Individual State dataframe setup

#### Create a dataframe without null values and does not contain state localities

In [ ]:
confirm_US_df = confirm_df[~confirm_df['Province/State'].isnull()].copy()

confirm_US_df = confirm_US_df[~confirm_US_df['Province/State'].str.contains(',') & (confirm_US_df['Country/Region']=='US')].sort_values('Province/State').copy()

confirm_US_df = confirm_US_df.rename(columns={'Province/State': 'state'})
confirm_US_df_timeline = confirm_US_df.copy()
confirm_US_df_timeline.head()

#### Simplify the datafram containing the US confirmed cases

In [ ]:
num_columns = len(confirm_US_df.columns)

confirm_US_df['confirmed'] = confirm_US_df[confirm_US_df.columns[4: num_columns]].max(axis=1).astype(int)
confirm_US_df = confirm_US_df[['state','Lat','Long','confirmed']].copy()
#sort descending by total count
confirm_US_df[['state','confirmed']].sort_values(by= ['confirmed'], ascending=False).style.hide_index()

In [ ]:
confirm_US_df['confirmed'].sum()

#### Create a dataframe without null values and does not contain state localities

In [ ]:
recover_US_df = recover_df[~recover_df['Province/State'].isnull()].copy()

recover_US_df = recover_US_df[~recover_US_df['Province/State'].str.contains(',') & (recover_US_df['Country/Region']=='US')].sort_values('Province/State').copy()

recover_US_df = recover_US_df.rename(columns={'Province/State': 'state'})
recover_US_df_timeline = recover_US_df.copy()
recover_US_df_timeline.head()

#### Simplify the dataframe containing the US `recovered` cases

In [ ]:
num_columns = len(recover_US_df.columns)

recover_US_df['recovered'] = recover_US_df[recover_US_df.columns[4: num_columns]].max(axis=1).astype(int)
recover_US_df = recover_US_df[['state','Lat','Long','recovered']].copy()
#sort descending by total count
recover_US_df[['state','recovered']].sort_values(by= ['recovered'], ascending=False).style.hide_index()

In [ ]:
recover_US_df['recovered'].sum()

#### Create a dataframe without null values and does not contain state localities

In [ ]:
deaths_US_df = deaths_df[~deaths_df['Province/State'].isnull()].copy()

deaths_US_df = deaths_US_df[~deaths_US_df['Province/State'].str.contains(',') & (deaths_US_df['Country/Region']=='US')].sort_values('Province/State').copy()

deaths_US_df = deaths_US_df.rename(columns={'Province/State': 'state'})
deaths_US_df_timeline = deaths_US_df.copy()
deaths_US_df_timeline.head()

#### Simplify the datafram containing the US `deaths` cases

In [ ]:
num_columns = len(deaths_US_df.columns)

deaths_US_df['deaths'] = deaths_US_df[deaths_US_df.columns[4: num_columns]].max(axis=1).astype(int)
deaths_US_df = deaths_US_df[['state','Lat','Long','deaths']].copy()
#sort descending by total count
deaths_US_df[['state','deaths']].sort_values(by= ['deaths'], ascending=False).style.hide_index()

In [ ]:
deaths_US_df['deaths'].sum()

In [ ]:
states_df = pd.merge(confirm_US_df, recover_US_df, on='state')
#states_df

states_df = pd.merge(states_df, deaths_US_df, on='state')
states_df = states_df[['state','Lat','Long','confirmed','recovered','deaths']]
#states_df

#### Create a dataframe of current fips codes for counties and states

In [ ]:
county_fips_url = 'https://api.census.gov/data/2010/dec/sf1?get=NAME&for=county:*'
state_fips_url = 'https://api.census.gov/data/2010/dec/sf1?get=NAME&for=state:*'

# create county with fips dataframe
r = requests.get(county_fips_url)
county_fips_df = pd.DataFrame(r.json())
#convert the first row to the header
new_header_county = county_fips_df.iloc[0] #grab the first row for the header
county_fips_df = county_fips_df[1:] #take the data less the header row
county_fips_df.columns = new_header_county #set the header row as the df header
print(len(county_fips_df))
# create state with fips dataframe
r = requests.get(state_fips_url)
state_fips_df = pd.DataFrame(r.json())
#convert the first row to the header
new_header_state = state_fips_df.iloc[0] #grab the first row for the header
state_fips_df = state_fips_df[1:] #take the data less the header row
state_fips_df.columns = new_header_state #set the header row as the df header
state_fips_df=pd.merge(state_fips_df, state_abbr_fips_df, on='NAME')
print(len(state_fips_df))


fips_df = pd.merge(county_fips_df, state_fips_df, on='state')

fips_df = fips_df.rename(columns={'NAME_x': 'full_county', 'county': 'county_fips'})

fips_df['county']=fips_df.full_county.str.split(",",expand=True)[0]

fips_df['county_short']=fips_df.county.str.split(" County",expand=True)[0]


fips_df=fips_df[['state_fips','county_fips','state','county','county_short','full_county','state_abbr']].sort_values(by=['state','county']).copy()
fips_df.reset_index(inplace = True,drop=True) 
print(len(fips_df))

fips_df


In [ ]:
states_df = pd.merge(confirm_US_df, recover_US_df, how='left',left_on='state',right_on='state')
#states_df

states_df = pd.merge(states_df, deaths_US_df, how='left',left_on='state',right_on='state')
states_df = states_df[['state','Lat','Long','confirmed','recovered','deaths']]
print(len(states_df))
#states_df

In [ ]:
states_df = pd.merge(states_df,state_fips_df, left_on='state', right_on='NAME')
states_df = states_df.rename(columns={'state_x': 'state'})
states_df = states_df[['state','state_fips','state_abbr','Lat','Long','confirmed','recovered','deaths']]
print(len(states_df))
states_df.sort_values(by='confirmed',ascending=False).head()

In [ ]:
confirm_counties_df = confirm_df[~confirm_df['Province/State'].isnull()].copy()
confirm_counties_df = confirm_counties_df[confirm_counties_df['Province/State'].str.contains(',') & (confirm_counties_df['Country/Region']=='US')].sort_values('Province/State').copy()
confirm_counties_df = confirm_counties_df.rename(columns={'Province/State': 'county_state'})
confirm_counties_df_timeline = confirm_counties_df.copy()
print(len(confirm_counties_df))
confirm_counties_df_timeline.head()

In [ ]:
num_columns = len(confirm_counties_df.columns)
confirm_counties_df['confirmed'] = confirm_counties_df[confirm_counties_df.columns[4: num_columns]].max(axis=1).astype(int)
confirm_counties_df=confirm_counties_df[['county_state','Lat','Long','confirmed']].copy()
split_county_state=confirm_counties_df.county_state.str.split(", ",expand=True)
confirm_counties_df['county']=split_county_state[0]
confirm_counties_df['state']=split_county_state[1]
confirm_counties_df=confirm_counties_df[['county_state','county','state','Lat','Long','confirmed']]
print(len(confirm_counties_df))

#confirm_counties_df.sort_values(by='confirmed',ascending=False)

In [ ]:
recover_counties_df = recover_df[~recover_df['Province/State'].isnull()].copy()
recover_counties_df = recover_counties_df[recover_counties_df['Province/State'].str.contains(',') & (recover_counties_df['Country/Region']=='US')].sort_values('Province/State').copy()
recover_counties_df = recover_counties_df.rename(columns={'Province/State': 'county_state'})
recover_counties_df_timeline = recover_counties_df.copy()
print(len(recover_counties_df))

recover_counties_df_timeline.head()

In [ ]:
num_columns = len(recover_counties_df.columns)
recover_counties_df['recovered'] = recover_counties_df[recover_counties_df.columns[4: num_columns]].max(axis=1).astype(int)
recover_counties_df=recover_counties_df[['county_state','Lat','Long','recovered']].copy()
split_county_state=recover_counties_df.county_state.str.split(", ",expand=True)
recover_counties_df['county']=split_county_state[0]
recover_counties_df['state']=split_county_state[1]
recover_counties_df=recover_counties_df[['county_state','county','state','Lat','Long','recovered']]
print(len(recover_counties_df))

#recover_counties_df.sort_values(by='recovered',ascending=False)

In [ ]:
deaths_counties_df = deaths_df[~deaths_df['Province/State'].isnull()].copy()
deaths_counties_df = deaths_counties_df[deaths_counties_df['Province/State'].str.contains(',') & (deaths_counties_df['Country/Region']=='US')].sort_values('Province/State').copy()
deaths_counties_df = deaths_counties_df.rename(columns={'Province/State': 'county_state'})
deaths_counties_df = deaths_counties_df.copy()
print(len(deaths_counties_df))

#deaths_counties_df.head()

In [ ]:
num_columns = len(deaths_counties_df.columns)
deaths_counties_df['deaths'] = deaths_counties_df[deaths_counties_df.columns[4: num_columns]].max(axis=1).astype(int)
deaths_counties_df=deaths_counties_df[['county_state','Lat','Long','deaths']].copy()
split_county_state=deaths_counties_df.county_state.str.split(", ",expand=True)
deaths_counties_df['county']=split_county_state[0]
deaths_counties_df['state']=split_county_state[1]

deaths_counties_df=deaths_counties_df[['county_state','county','state','Lat','Long','deaths']]
print(len(deaths_counties_df))
#deaths_counties_df.sort_values(by='deaths',ascending=False)

In [ ]:
counties_df = pd.merge(confirm_counties_df, recover_counties_df, how='left', on=['state','county'])
counties_df = pd.merge(counties_df, deaths_counties_df, on=['state','county'])
counties_df = counties_df[['county_state','county','state','Lat','Long','confirmed','recovered','deaths']]


counties_df['county_short']=counties_df.county.str.split(" County",expand=True)[0]
print(len(counties_df))

#counties_df

In [ ]:
counties_df = pd.merge(counties_df,fips_df, how='left', left_on=['county_short','state'], right_on=['county_short','state_abbr'])
#new_df = pd.merge(A_df, B_df,  how='left', left_on=['A_c1','c2'], right_on = ['B_c1','c2'])
print(len(counties_df))
#counties_df

In [ ]:
counties_df = counties_df.rename(columns={'state_x': 'state','county_x':'county'})
counties_df = counties_df[['county_state','county','state','state_fips','county_fips','county_short','full_county','Lat','Long','confirmed','recovered','deaths']]
counties_df = counties_df.sort_values(by=['confirmed','state_fips','county_fips'],ascending=False)

counties_df.head(15)

In [ ]:
len(counties_df[counties_df.state=='NY'])

In [ ]:
counties_df[counties_df.state=='KY']

In [ ]:
states_df_plot=states_df[['state','confirmed']].sort_values(by=['confirmed'],ascending=False).copy().head(10)

states_df_plot.reset_index(drop=True, inplace=True)
states_df_plot.head()

In [ ]:
states_df_plot=states_df_plot.set_index('state').copy()
states_df_plot[['confirmed']].head(10).plot(kind='barh').invert_yaxis()

In [ ]:
states_df_plot

In [ ]:
states_df_plot.columns

In [ ]:
%matplotlib --list

In [ ]:
fig = plt.figure(figsize=(50,50))
plt.rcParams["axes.labelsize"] = 15


In [ ]:
states_df_plot2=sns.catplot(x=list(states_df_plot.confirmed),y=list(states_df_plot.index),kind='bar',data=states_df_plot,edgecolor='whitesmoke',linewidth=.5,color='blue')
states_df_plot2.set(xlabel='confirmed cases',ylabel='')
for bar_index, bar_attributes in enumerate(states_df_plot2.ax.patches):
    bar_width = bar_attributes.get_width() # x coordinate of text
    states_df_plot2.ax.text(
        bar_width+35, bar_index, '{}'.format(int(bar_width)),
        ha='center', va='center',size=14, color='red')
plt.show;
